In [ ]:
from scapy.all import *
import os

def analyze_qos(pcap_file):
    print(f"[*] Menganalisis file: {pcap_file} ...")
    
    # Cek apakah file ada
    if not os.path.exists(pcap_file):
        print(f"[!] File {pcap_file} tidak ditemukan.")
        return

    # Baca file PCAP (Hati-hati, file besar bisa memakan RAM)
    # Untuk file sangat besar, gunakan PcapReader (iterative)
    try:
        packets = rdpcap(pcap_file)
    except Exception as e:
        print(f"[Error] Gagal membaca pcap: {e}")
        return

    if len(packets) == 0:
        print("[!] File PCAP kosong.")
        return

    # --- 1. MENGHITUNG DURASI ---
    # Ambil timestamp paket pertama dan terakhir
    start_time = packets[0].time
    end_time = packets[-1].time
    duration = end_time - start_time
    
    if duration == 0:
        duration = 0.0001 # Hindari pembagian dengan nol

    # --- 2. MENGHITUNG THROUGHPUT ---
    # Total ukuran seluruh paket (dalam bytes)
    total_bytes = sum(len(pkt) for pkt in packets)
    # Konversi ke bits (1 byte = 8 bits)
    total_bits = total_bytes * 8
    # Throughput (bps) = Total Bits / Durasi
    throughput_bps = total_bits / duration
    throughput_kbps = throughput_bps / 1000
    throughput_mbps = throughput_kbps / 1000

    # --- 3. ESTIMASI PACKET LOSS (TCP RETRANSMISSION) ---
    # Logika sederhana: Jika Sequence Number TCP muncul dua kali, itu retransmission.
    tcp_retransmissions = 0
    seen_seq = set()
    total_tcp = 0

    for pkt in packets:
        if TCP in pkt:
            total_tcp += 1
            # Kunci unik: Source IP, Dest IP, dan Sequence Number
            # Jika kunci ini sudah ada sebelumnya, kemungkinan ini paket kirim ulang (loss)
            key = (pkt[IP].src, pkt[IP].dst, pkt[TCP].seq)
            
            if key in seen_seq:
                tcp_retransmissions += 1
            else:
                seen_seq.add(key)
    
    # Hitung persentase loss (dari sudut pandang retransmission)
    if total_tcp > 0:
        loss_rate = (tcp_retransmissions / total_tcp) * 100
    else:
        loss_rate = 0

    # --- PRINT HASIL ---
    print("\n" + "="*40)
    print(f"HASIL ANALISIS QoS: {pcap_file}")
    print("="*40)
    print(f"Total Paket       : {len(packets)}")
    print(f"Durasi Capture    : {duration:.2f} detik")
    print(f"Total Data        : {total_bytes / 1024:.2f} KB")
    print("-" * 30)
    print(f"Throughput        : {throughput_bps:.2f} bps")
    print(f"                  : {throughput_kbps:.2f} Kbps")
    print(f"                  : {throughput_mbps:.2f} Mbps")
    print("-" * 30)
    print(f"Total TCP Packets : {total_tcp}")
    print(f"Retransmissions   : {tcp_retransmissions}")
    print(f"Est. Packet Loss  : {loss_rate:.4f} %")
    print("="*40 + "\n")

# Ganti nama file sesuai dengan file pcap yang sudah Anda copy ke laptop
# Contoh: 'inline-out.pcap' atau file hasil copy Anda
if __name__ == "__main__":
    # Ganti path ini ke lokasi file pcap di laptop Anda
    target_file = "inline-out.pcap" 
    analyze_qos(target_file)